### Bag of Words Model for the OHSUmed corpus
#### Establishing a baseline in Keras

The OHSUmed test collection is a subset of the MEDLINE database, which is a bibliographic database of important, peer-reviewed medical literature maintained by the National Library of Medicine. The subset we consider is the collection consisting of the first 20,000 documents from the 50,216 medical abstracts of the year 1991. The classification scheme consists of the 23 Medical Subject Headings (MeSH) categories of cardiovascular diseases group. 

After selecting such category subset, the document number is 13,924 documents (6,285 for training and 7,649 for testing). Of the 23 categories of the cardiovascular diseases group

In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from src.models import metrics
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

# incorporate only frequent labels
def isolate_frequent_labels(X, label_column, threshold_count):
    # returns: dataframe with only infrequent labels
    df = X.groupby(label_column).size()[X.groupby(label_column).size() > threshold_count].reset_index()
    frequent_labels = df.iloc[:,0]
    return X[X.label.isin(frequent_labels)]

OHSUcsv = pd.read_csv("../data/processed/ohsumed_abstracts.csv", index_col ="Unnamed: 0")
data = isolate_frequent_labels(OHSUcsv, 'label', 200)
#data = OHSUcsv[OHSUcsv.label.isin(frequent_labels)]

train_posts = data.loc[data.split == 'train', 'doc']
train_tags = data.loc[data.split == 'train', 'label']
test_posts = data.loc[data.split == 'test', 'doc']
test_tags = data.loc[data.split == 'test', 'label']

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Preprocessing with Keras
### Text Input
Keras has some built in methods for preprocessing text to make preprocessing simple. 

##### Tokenizer class
The Tokenizer class provides methods to count the unique words in our vocabulary and assign each of those words to indices. We’ll create an instance of the Tokenizer class, and then pass it the Pandas dataframe of text we want to train on. Although the Tokenizer function, takes a 'num_words' argument to limit the text to a certain vocabulary size (say, 10,000), we will instead use all words. Note that stopwords were already removed.

##### fit_on_texts
Calling fit_on_texts() automatically creates a word index lookup of our vocabulary, thereby associating each word with a unique number.  

##### texts_to_matrix
With our Tokenizer, we can now use the texts_to_matrix method to create the training data we’ll pass our model. This will take each post’s text and turn it into a vocab_size “bag” array, with 1s indicating the indices where words in a question are present in the vocabulary. 

In [2]:
vocab_size = 10000

tokenize = text.Tokenizer(num_words = vocab_size)
tokenize.fit_on_texts(train_posts)
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

### Preprocessing output labels
The tag for each question is a number (i.e. “1” or “2”). Instead of using a single int as the label for each input, we’ll turn it into a one-hot vector. **We feed a one-hot vector to our model instead of a single integer because the models will output a vector of probabilities for each document** 

scikit-learn has a **LabelBinarizer class** which makes it easy to build these one-hot vectors. We can pass it the labels column from our Pandas DataFrame and then call fit() and transform() on it:

In [3]:
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

## Building a simple multilayer perception
### Using the Sequential Model API in Keras
To define the layers of our model we’ll use the Keras **Sequential model API**, which composes a linear stack of layers.

**1st layer**: The input layer will take the vocab_size arrays for each comment. We’ll specify this as a Dense layer in Keras, which means each neuron in this layer will be fully connected to all neurons in the next layer. We pass the Dense layer two parameters: the dimensionality of the layer’s output (number of neurons) and the shape of our input data. Choosing the number of dimensions requires some experimentation, but most use a power of 2 as the number of dimensions, so we’ll start with 512.

**2nd Layer**: The final layer will use the Softmax activation function, which normalizes the evidence for each possible label into a probability (from 0 to 1). For each category, there is a True and False label, so we have 2 units. If we were to allow our model to model the probability of all N (mutually-exclusive) categories, this final layer would have N+1 units. 

**Training Protocol**: We call the compile method with the loss function we want to use, the type of optimizer, and the metrics our model should evaluate during training and testing. We’ll use the cross entropy loss function, since each of our abstrats can only belong to one post. The optimizer is the function our model uses to minimize loss. In this example we’ll use the Adam optimizer. There are many optimizers available, all of which are different implementations of gradient descent. For metrics we’ll evaluate accuracy, which will tell us the percentage of abstracts assigned the correct label.

In [4]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dense(units=2, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['binary_accuracy'])

## Training the Model
### Assigning a Cost Function and Optimizer


To train our model, we’ll call the fit() method, pass it our training data and labels, the number of examples to process in each batch (batch size), how many times the model should train on our entire dataset (epochs), and the validation split. validation_split tells Keras what percentage of our training data to reserve for validation.

**Try tweaking these hyperparameters when using this model on our own data!**

In [26]:
# try to predict only the first label
column_index = 1

label_train = y_train[:,column_index]
history = model.fit(x_train,  label_train, 
                    batch_size=32, 
                    epochs=1, 
                    verbose=1,
                    validation_split=0.1)

y_pred = model.predict(x_test, verbose=1)

Train on 5465 samples, validate on 608 samples
Epoch 1/1
7367/7367 [==============================] - 1s 150us/step


In [25]:
print(metrics.get_binary_results(y_pred, y_test, 1))

returns: dataframe with precision, recall, and F
   precision    recall        f1
0   0.001735  0.021739  0.003213


### Evaluation

2.3514701e-09

In [14]:
y_test[:,0]

7367